In [4]:
%111pip install html5lib
%pip install webdriver-manager
%pip install selenium
"""这是一个多行注释（文档字符串）
可以用来为整个Python文件、类或函数提供详细说明
通常放在文件开头、类定义或函数定义的开始位置
"""
#11
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re
## 房价信息爬取
%pip install html5lib
%pip install webdriver-manager
%pip install selenium
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, TimeoutException
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import re

## 房价信息爬取
# 设置WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 目标URL
base_url = "https://lf.esf.fang.com/house-a010278/"
# 存储所有房源信息
all_properties = []

try:
    # 遍历前20页
    for page in range(1, 21):
        if page == 1:
            url = base_url
        else:
            url = f"{base_url}i3{page}/"
        
        print(f"正在爬取第{page}页: {url}")
        driver.get(url)
        
        # 等待页面加载完成
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, "shop_list"))
        )
        
        # 修复：使用CSS选择器而不是复合类名
        property_list = driver.find_elements(By.CSS_SELECTOR, ".shop_list dl")
        print(f"找到{len(property_list)}个房源")
        
        # 提取每个房源的信息
        for property_item in property_list:
            try:
                # 初始化字典存储房源信息
                property_info = {}
                
                # 提取标题和链接
                title_element = property_item.find_element(By.CLASS_NAME, "tit_shop")
                property_info['标题'] = title_element.text
                
                # 提取价格信息
                try:
                    price_element = property_item.find_element(By.CLASS_NAME, "price_right")
                    price_text = price_element.text
                    property_info['总价'] = price_text.split('万')[0] + '万' if '万' in price_text else price_text
                    
                    # 尝试提取单价
                    try:
                        unit_price_element = price_element.find_element(By.TAG_NAME, "span")
                        property_info['单价'] = unit_price_element.text
                    except NoSuchElementException:
                        property_info['单价'] = "未知"
                except NoSuchElementException:
                    property_info['总价'] = "未知"
                    property_info['单价'] = "未知"
                
                # 提取房屋基本信息
                try:
                    tel_shop = property_item.find_element(By.CLASS_NAME, "tel_shop")
                    details = tel_shop.text.split('|')
                    
                    # 解析户型、面积、朝向等信息
                    for detail in details:
                        detail = detail.strip()
                        if '室' in detail and '厅' in detail:
                            property_info['户型'] = detail
                        elif '㎡' in detail:
                            property_info['面积'] = detail
                        elif '向' in detail:
                            property_info['朝向'] = detail
                        elif '层' in detail:
                            property_info['楼层'] = detail
                        elif '年建' in detail:
                            property_info['建成年份'] = detail
                except NoSuchElementException:
                    pass
                
                # 提取小区名称和地址
                try:
                    add_shop = property_item.find_element(By.CLASS_NAME, "add_shop")
                    address_parts = add_shop.text.split('\n')
                    if len(address_parts) >= 1:
                        property_info['小区名称'] = address_parts[0]
                    if len(address_parts) >= 2:
                        property_info['地址'] = address_parts[1]
                except NoSuchElementException:
                    pass
                
                # 提取特色标签
                try:
                    label_elements = property_item.find_elements(By.CLASS_NAME, "label")
                    labels = [label.text for label in label_elements if label.text]
                    property_info['特色标签'] = '|'.join(labels)
                except NoSuchElementException:
                    property_info['特色标签'] = ""
                
                # 提取经纪人信息
                try:
                    people_element = property_item.find_element(By.CLASS_NAME, "people_name")
                    property_info['经纪人'] = people_element.text
                except NoSuchElementException:
                    property_info['经纪人'] = "未知"
                
                # 提取交通信息
                try:
                    # 尝试查找交通信息，可能需要进一步解析
                    traffic_info = property_item.find_element(By.XPATH, ".//*[contains(text(), '距')]")
                    property_info['交通信息'] = traffic_info.text
                except NoSuchElementException:
                    property_info['交通信息'] = "未知"
                
                # 将房源信息添加到列表
                all_properties.append(property_info)
                
            except Exception as e:
                print(f"提取房源信息时出错: {e}")
                continue
        
        # 等待一段时间再请求下一页，避免过于频繁
        time.sleep(2)
        
except Exception as e:
    print(f"爬取过程中出错: {e}")
finally:
    driver.quit()

# 将数据转换为DataFrame
df = pd.DataFrame(all_properties)
# 保存到CSV文件
df.to_csv('Housing_Price_Dachang.csv', index=False, encoding='utf-8-sig')
print(f"爬取完成，共获取{len(all_properties)}条房源信息，已保存到Housing_Price_Dachang.csv")
## 租价信息爬取
# 设置WebDriver
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(service=service)

# 目标URL
base_url_rent = "https://lf.zu.fang.com/house-a010278/"
# 存储所有房源信息
all_properties = []

try:
    # 遍历前20页
    for page in range(1, 21):
        if page == 1:
            url = base_url_rent
        else:
            url = f"{base_url_rent}i3{page}/"
        
        print(f"正在爬取第{page}页: {url}")
        driver.get(url)
        
        # 等待页面加载完成
        WebDriverWait(driver, 15).until(
            EC.presence_of_element_located((By.CLASS_NAME, "houseList"))
        )
        
        # 获取所有房源元素 - 根据图片2中的HTML结构
        property_list = driver.find_elements(By.CSS_SELECTOR, ".houseList > .list")
        print(f"找到{len(property_list)}个房源")
        
        # 提取每个房源的信息
        for index, property_item in enumerate(property_list):
            try:
                # 初始化字典存储房源信息
                property_info = {}
                property_info['页码'] = page
                
                # 提取标题和链接 - 根据图片2中的HTML结构
                try:
                    title_element = property_item.find_element(By.CSS_SELECTOR, ".title a")
                    property_info['标题'] = title_element.text.strip()
                    property_info['链接'] = title_element.get_attribute('href')
                except NoSuchElementException:
                    print(f"第{page}页第{index+1}个房源缺少标题信息")
                    continue
                
                # 提取价格信息 - 根据图片1中的信息
                try:
                    # 尝试查找价格元素，可能类名为"money"或其他
                    price_element = property_item.find_element(By.CSS_SELECTOR, ".money")
                    property_info['月租金'] = price_element.text.strip()
                except NoSuchElementException:
                    # 如果找不到特定类名，尝试通过文本内容查找
                    try:
                        price_text = property_item.find_element(By.XPATH, ".//*[contains(text(), '元/月')]")
                        property_info['月租金'] = price_text.text.strip()
                    except:
                        property_info['月租金'] = "未知"
                
                # 提取房屋基本信息 - 根据图片2中的HTML结构
                try:
                    info_element = property_item.find_element(By.CLASS_NAME, "font15")
                    info_text = info_element.text
                    
                    # 解析租赁类型、户型、面积、朝向等信息
                    parts = info_text.split('|')
                    for part in parts:
                        part = part.strip()
                        if '整租' in part or '合租' in part:
                            property_info['租赁类型'] = part
                        elif '室' in part and '厅' in part:
                            property_info['户型'] = part
                        elif '㎡' in part:
                            property_info['面积'] = part
                        elif '朝' in part:
                            property_info['朝向'] = part
                except NoSuchElementException:
                    pass
                
                # 提取位置信息 - 根据图片1中的信息
                try:
                    # 尝试查找位置元素，可能类名为"gray6"或其他
                    location_element = property_item.find_element(By.CLASS_NAME, "gray6")
                    location_text = location_element.text.strip()
                    
                    # 解析区域-商圈-小区信息
                    if '-' in location_text:
                        parts = location_text.split('-')
                        if len(parts) >= 3:
                            property_info['区域'] = parts[0]
                            property_info['商圈'] = parts[1]
                            property_info['小区'] = parts[2]
                        elif len(parts) == 2:
                            property_info['区域'] = parts[0]
                            property_info['商圈'] = parts[1]
                    else:
                        property_info['位置'] = location_text
                except NoSuchElementException:
                    pass
                
                # 提取交通信息 - 根据图片1中的信息
                try:
                    # 尝试查找包含"距"字的元素
                    traffic_element = property_item.find_element(By.XPATH, ".//*[contains(text(), '距')]")
                    property_info['交通信息'] = traffic_element.text.strip()
                except NoSuchElementException:
                    property_info['交通信息'] = "未知"
                
                # 提取特色标签 - 根据图片2中的HTML结构
                try:
                    # 尝试查找标签元素，可能类名为"note"或其他
                    tag_elements = property_item.find_elements(By.CLASS_NAME, "note")
                    tags = [tag.text for tag in tag_elements if tag.text.strip()]
                    property_info['特色标签'] = '|'.join(tags)
                except NoSuchElementException:
                    property_info['特色标签'] = ""
                
                # 将房源信息添加到列表
                all_properties.append(property_info)
                print(f"已提取第{page}页第{index+1}个房源: {property_info['标题']}")
                
            except Exception as e:
                print(f"提取第{page}页第{index+1}个房源信息时出错: {e}")
                continue
        
        # 等待一段时间再请求下一页，避免过于频繁
        time.sleep(2)
        
except Exception as e:
    print(f"爬取过程中出错: {e}")
finally:
    driver.quit()

# 将数据转换为DataFrame
df = pd.DataFrame(all_properties)

# 保存到CSV文件
df.to_csv('Housing_Rent_Dachang.csv', index=False, encoding='utf-8-sig')
print(f"爬取完成，共获取{len(all_properties)}条房源信息，已保存到fangtianxia_rental_properties.csv")


Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
正在爬取第1页: https://lf.esf.fang.com/house-a010278/
找到60个房源
正在爬取第2页: https://lf.esf.fang.com/house-a010278/i32/
找到60个房源
正在爬取第3页: https://lf.esf.fang.com/house-a010278/i33/
找到60个房源
正在爬取第4页: https://lf.esf.fang.com/house-a010278/i34/
找到60个房源
正在爬取第5页: https://lf.esf.fang.com/house-a010278/i35/
找到60个房源
正在爬取第6页: https://lf.esf.fang.com/house-a010278/i36/
找到60个房源
正在爬取第7页: https://lf.esf.fang.com/house-a010278/i37/
找到60个房源
正在爬取第8页: https://lf.esf.fang.com/house-a010278/i38/
找到60个房源
正在爬取第9页: https://lf.esf.fang.com/house-a010278/i39/
找到60个房源
正在爬取第10页: https://lf.esf.fang.com/house-a010278/i310/
找到60个房源
正在爬取第1